In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc, roc_auc_score
)

models_folder = "Modelos"
results_folder = "Resultados"
predictions_folder = "Predicciones"

# Creamos carpetas si no existen
for folder in [results_folder, predictions_folder]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Mapeo de datasets (igual que en train.ipynb)
datasets_map = {
    "original": {"folder": "Original", "suffix": ""},
    "originalPCA95": {"folder": "Original", "suffix": "_pca95"},
    "originalPCA80": {"folder": "Original", "suffix": "_pca80"},
    "estandarizado": {"folder": "Estandarizado", "suffix": ""},
    "estandarizadoPCA95": {"folder": "Estandarizado", "suffix": "_pca95"},
    "estandarizadoPCA80": {"folder": "Estandarizado", "suffix": "_pca80"},
    "normalizado": {"folder": "Normalizado", "suffix": ""},
    "normalizadoPCA95": {"folder": "Normalizado", "suffix": "_pca95"},
    "normalizadoPCA80": {"folder": "Normalizado", "suffix": "_pca80"},
}

methods = ["knn", "svm", "nb", "rf"]
folds = [1, 2, 3, 4, 5]

Vamos a crear una función auxiliar que calcule todo esto a partir de las etiquetas reales (y_true) y las predicciones (y_pred).

Nota Importante sobre Multiclase: El dataset Iris tiene 3 clases. Las métricas como ROC/AUC y Especificidad se definen típicamente para problemas binarios (Sí/No). Para resolver esto en multiclase, usaremos el promedio "macro" (calcular para cada clase y promediar) o "weighted". El código abajo está preparado para manejar esto automáticamente.

In [2]:
def calculate_metrics(y_true, y_pred, y_proba):
   
    # Esta función calcula todas las métricas solicitadas
   

    cm = confusion_matrix(y_true, y_pred)
    
    # Como es un problema multiclase; TP, TN, FP, FN se calculan con el uno contra todos (se explica en el informe)
    
    FP = cm.sum(axis=0) - np.diag(cm)  
    print(f"{FP}\n")
    FN = cm.sum(axis=1) - np.diag(cm)
    print(f"{FN}\n")
    TP = np.diag(cm)
    print(f"{TP}\n")
    TN = cm.sum() - (FP + FN + TP)
    print(f"{TN}\n")

    # Convertimos a float para las divisiones
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    # --- Métricas ---
    # Sensibilidad (TPR, Recall)         
    Sensibilidad = TP / (TP + FN)
    # Especificidad (TNR)
    Especificidad = TN / (TN + FP)
    # Tasa Falsos Positivos
    FPR = FP / (FP + TN)
    # Tasa Falsos Negativos
    FNR = FN / (TP + FN)
    # Precisión
    PR = TP / (TP + FP)
    # Recall
    RC = TP / (TP + FN)
    # F1-Score
    FM = 2 * (PR * RC) / (PR + RC)
    # Exactitud
    AC = (TP + TN) / (TP + TN + FP + FN)

    
    # --- Promedios globales ---
    # Usamos np.nanmean para evitar errores si alguna clase no tiene muestras (div/0)
    metrics = {
        "Accuracy": np.nanmean(AC),
        "Precision": np.nanmean(PR),
        "Recall": np.nanmean(RC) , # Igual a Sensibilidad media
        "F1-Score": np.nanmean(FM),
        "Sensitivity": np.nanmean(Sensibilidad),
        "Specificity": np.nanmean(Especificidad),
        "FPR": np.nanmean(FPR),
        "FNR": np.nanmean(FNR)
    }
    
    # --- AUC (Area Under Curve) ---
    try:
        if y_proba.shape[1] == 2: 
             auc_score = roc_auc_score(y_true, y_proba[:, 1])
        else: 
             auc_score = roc_auc_score(y_true, y_proba, multi_class='ovr', average='macro')
    except:
        auc_score = 0.0 
        
    metrics["AUC"] = auc_score
    
    return metrics

Ahora creamos el bucle principal. Este código hará lo siguiente:

Cargará el modelo .pkl correspondiente.

Cargará el test.csv correspondiente (datos que el modelo NO vio).

Generará predicciones de clase (predict) y probabilidades (predict_proba).

Guardará estas predicciones en un archivo CSV en la carpeta predictions/. Esto es vital para la parte de Ensemble.

Calculará las métricas y las irá guardando en una lista gigante.

In [3]:
# Lista para acumular todos los resultados
all_results = []

print("Iniciando evaluación ...")

for fold in folds:
    for ds_name, ds_info in datasets_map.items():
        # Cargamos los datos de TEST para este fold/dataset
        test_filename = f"test{fold}{ds_info['suffix']}.csv"
        test_path = os.path.join(ds_info['folder'], test_filename)
        
        if not os.path.exists(test_path):
            print(f"Skip: No datos test para {ds_name} fold {fold}")
            continue
            
        data_test = pd.read_csv(test_path, header=None)
        X_test = data_test.iloc[:, :-1].values
        y_test = data_test.iloc[:, -1].values.ravel()
        
        for method in methods:
            # Cargamos el modelo
            model_name = f"model_{fold}_{ds_name}_{method}.pkl"
            model_path = os.path.join(models_folder, model_name)
            
            if not os.path.exists(model_path):
                print(f"Skip: No modelo {model_name}")
                continue
                
            clf = joblib.load(model_path)
            
            # Predecir
            y_pred = clf.predict(X_test)
            
            # Intentamos obtener las probabilidades
            if hasattr(clf, "predict_proba"):
                y_proba = clf.predict_proba(X_test)
            else:
                n_classes = len(np.unique(y_test))
                y_proba = np.zeros((len(y_test), n_classes))
                for i, pred in enumerate(y_pred):
                    y_proba[i, int(pred)] = 1.0

            # Guardamos las predicciones para la posterior combinación (Ensemble)
            pred_df = pd.DataFrame()
            pred_df['y_true'] = y_test
            pred_df['y_pred'] = y_pred
            # Añadimos las columnas de probabilidad
            for c in range(y_proba.shape[1]):
                pred_df[f'prob_{c}'] = y_proba[:, c]
            
            pred_filename = f"pred_{fold}_{ds_name}_{method}.csv"
            pred_df.to_csv(os.path.join(predictions_folder, pred_filename), index=False)

            # Calculamos las métricas
            metrics = calculate_metrics(y_test, y_pred, y_proba)
            
        
            metrics['Fold'] = fold
            metrics['Method'] = method
            metrics['Dataset'] = ds_name
            
            all_results.append(metrics)

# --- Guardar resultados globales ---
results_df = pd.DataFrame(all_results)
# Guardamos el CSV con todas las métricas de todas las iteraciones
results_csv_path = os.path.join(results_folder, "all_metrics_raw.csv")
results_df.to_csv(results_csv_path, index=False)

print(f"\nEvaluación finalizada")
print(f"  Predicciones guardadas en: '{predictions_folder}/'")
print(f"  Tabla métricas guardada en: '{results_csv_path}'")
print(f"  Total evaluaciones: {len(results_df)}")

Iniciando evaluación ...
[0 1 0]

[0 0 1]

[10 10  9]

[20 19 20]

[0 1 0]

[0 0 1]

[10 10  9]

[20 19 20]

[0 1 1]

[0 1 1]

[10  9  9]

[20 19 19]

[0 1 0]

[0 0 1]

[10 10  9]

[20 19 20]

[0 1 0]

[0 0 1]

[10 10  9]

[20 19 20]

[0 1 1]

[0 1 1]

[10  9  9]

[20 19 19]

[0 2 3]

[0 3 2]

[10  7  8]

[20 18 17]

[0 1 0]

[0 0 1]

[10 10  9]

[20 19 20]

[0 1 3]

[0 3 1]

[10  7  9]

[20 19 17]

[0 1 2]

[0 2 1]

[10  8  9]

[20 19 18]

[0 1 2]

[0 2 1]

[10  8  9]

[20 19 18]

[0 1 2]

[0 2 1]

[10  8  9]

[20 19 18]

[0 1 0]

[0 0 1]

[10 10  9]

[20 19 20]

[0 1 0]

[0 0 1]

[10 10  9]

[20 19 20]

[0 1 1]

[0 1 1]

[10  9  9]

[20 19 19]

[0 1 0]

[0 0 1]

[10 10  9]

[20 19 20]

[0 1 4]

[0 4 1]

[10  6  9]

[20 19 16]

[0 1 3]

[0 3 1]

[10  7  9]

[20 19 17]

[0 2 2]

[0 2 2]

[10  8  8]

[20 18 18]

[0 1 4]

[0 4 1]

[10  6  9]

[20 19 16]

[0 1 4]

[0 4 1]

[10  6  9]

[20 19 16]

[0 1 3]

[0 3 1]

[10  7  9]

[20 19 17]

[0 2 2]

[0 2 2]

[10  8  8]

[20 18 18]

[0 1 4]

